In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def create_gold_dataset(glycans_file, proteins_file, train_fractions_file, output_file):
    """
    Create a gold dataset by merging glycans, proteins, and train_fractions data.
    Performs one-hot encoding on categorical features.
    
    Parameters:
    -----------
    glycans_file : str
        Path to the detailed_glycans CSV file
    proteins_file : str
        Path to the detailed_proteins CSV file
    train_fractions_file : str
        Path to the train_fractions CSV file
    output_file : str
        Path to save the output gold_data CSV file
    
    Returns:
    --------
    pandas.DataFrame
        The merged gold dataset with one-hot encoded categorical features
    """
    print(f"Reading glycans data from {glycans_file}...")
    glycans = pd.read_csv(glycans_file)
    print(f"Read {len(glycans)} glycan records")
    
    print(f"Reading proteins data from {proteins_file}...")
    proteins = pd.read_csv(proteins_file)
    print(f"Read {len(proteins)} protein records")
    
    print(f"Reading train fractions data from {train_fractions_file}...")
    train_fractions = pd.read_csv(train_fractions_file, sep='\t')
    print(f"Read {len(train_fractions)} train fraction records")
    
    # Create maps for faster lookup
    glycans_map = {row['Name']: row for _, row in glycans.iterrows()}
    proteins_map = {row['ProteinGroup']: row for _, row in proteins.iterrows()}
    
    # Create the gold dataset by joining the data
    print("Merging datasets...")
    gold_data_records = []
    
    # Track how many records were successfully merged and how many had missing data
    total_records = len(train_fractions)
    missing_glycan = 0
    missing_protein = 0
    successful_merges = 0
    
    for _, fraction in train_fractions.iterrows():
        glycan_id = fraction['GlycanID']
        protein_group = fraction['ProteinGroup']
        
        # Get the corresponding glycan and protein records
        glycan = glycans_map.get(glycan_id)
        protein = proteins_map.get(protein_group)
        
        # Skip if either glycan or protein not found
        if glycan is None:
            missing_glycan += 1
            continue
            
        if protein is None:
            missing_protein += 1
            continue
        
        # Create a merged record
        merged_record = {
            # Base fields from train_fractions
            'ObjId': fraction['ObjId'],
            'ProteinGroup': fraction['ProteinGroup'],
            'Concentration': fraction['Concentration'],
            'GlycanID': fraction['GlycanID'],
            'f': fraction['f']
        }
        
        # Add prefix to glycan columns to avoid name conflicts
        for key, value in glycan.items():
            merged_record[f'Glycan_{key}'] = value
            
        # Add prefix to protein columns to avoid name conflicts
        for key, value in protein.items():
            merged_record[f'Protein_{key}'] = value
        
        gold_data_records.append(merged_record)
        successful_merges += 1
    
    # Create DataFrame from the merged records
    gold_data = pd.DataFrame(gold_data_records)

    # Remove specified columns
    columns_to_remove = ['GlycanID', 'Glycan_Name', 'Glycan_IUPAC', 'Glycan_SMILES', 'Glycan_GlycanClass', # Category columns
                         'Glycan_SlogP_VSA9', 'Glycan_SMR_VSA8', 'Glycan_PyranoseCount', 'Glycan_FuranoseCount', 'Glycan_BranchingPoints', # numerical columns
                         'ObjId', 'ProteinGroup'] # ID columns
    gold_data.drop(columns=columns_to_remove, inplace=True)
    
    # Print summary statistics
    print("\nMerge Summary:")
    print(f"Total train_fractions records: {total_records}")
    print(f"Records with missing glycans: {missing_glycan}")
    print(f"Records with missing proteins: {missing_protein}")
    print(f"Successfully merged records: {successful_merges}")
    
    # Perform one-hot encoding on categorical features
    print("\nPerforming one-hot encoding on categorical features...")

    
    # Combine all categorical columns
    categorical_columns = gold_data.select_dtypes(include=['object']).columns
    
    # Check if these columns exist in the dataframe
    existing_categorical_columns = [col for col in categorical_columns if col in gold_data.columns]
    
    if existing_categorical_columns:
        # Create a copy to preserve original data
        encoded_gold_data = gold_data.copy()
        
        # Handle each categorical column
        total_encoded_features = 0
        
        for col in existing_categorical_columns:
            # Fill NaN values with a placeholder
            encoded_gold_data[col] = encoded_gold_data[col].fillna('Unknown')
            
            # Get unique values for this categorical column
            unique_values = encoded_gold_data[col].unique()
            print(f"One-hot encoding {col} with {len(unique_values)} unique values")
            
            # Skip one-hot encoding if there's only one unique value
            if len(unique_values) <= 1:
                print(f"  Skipping {col} as it has only {len(unique_values)} unique value(s)")
                continue
                
            # Create one-hot encoder
            encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            
            # Fit and transform
            encoded_values = encoder.fit_transform(encoded_gold_data[[col]])
            
            # Create new column names for encoded features
            feature_names = [f"{col}_-_{val}" for val in encoder.categories_[0]]
            
            # Create a DataFrame with the encoded values
            encoded_df = pd.DataFrame(encoded_values, columns=feature_names, index=encoded_gold_data.index)
            
            # Concatenate the encoded values with the original DataFrame
            encoded_gold_data = pd.concat([encoded_gold_data, encoded_df], axis=1)
            
            # Remove the original categorical column
            encoded_gold_data.drop(col, axis=1, inplace=True)
            
            total_encoded_features += len(feature_names)
        
        # Update gold_data to the encoded version
        gold_data = encoded_gold_data
        print(f"One-hot encoding added {total_encoded_features} new features")
    else:
        print("No categorical columns found in the data. Skipping one-hot encoding.")
    
    # Save the gold dataset to CSV
    print(f"\nSaving gold dataset to {output_file}...")
    gold_data.to_csv(output_file, index=False)
    print(f"Gold dataset saved with {len(gold_data)} records and {len(gold_data.columns)} columns")
    
    return gold_data


In [2]:
# Define file paths
glycans_file = "detailed_glycans.csv"  
proteins_file = "detailed_proteins.csv"  
train_fractions_file = "Train_Fractions.csv"
output_file = os.path.join("..", "data", "gold_data_train.csv")

# Create the gold dataset
gold_data_train = create_gold_dataset(glycans_file, proteins_file, train_fractions_file, output_file)


Reading glycans data from detailed_glycans.csv...
Read 611 glycan records
Reading proteins data from detailed_proteins.csv...
Read 52 protein records
Reading train fractions data from Train_Fractions.csv...
Read 68492 train fraction records
Merging datasets...

Merge Summary:
Total train_fractions records: 68492
Records with missing glycans: 0
Records with missing proteins: 0
Successfully merged records: 68492

Performing one-hot encoding on categorical features...
One-hot encoding Protein_Accession with 37 unique values
One-hot encoding Protein_Uniprot with 37 unique values
One-hot encoding Protein_Description with 26 unique values
One-hot encoding Protein_Amino Acid Sequence with 37 unique values
One-hot encoding Protein_ProteinID with 37 unique values
One-hot encoding Protein_CleanSequence with 37 unique values
One-hot encoding Protein_PredictedSecondaryStructure with 3 unique values
One-hot encoding Protein_Stability with 2 unique values
One-hot encoding Protein_PredictedProteinCla

In [3]:
# Define file paths
glycans_file = "detailed_glycans.csv"  
proteins_file = "detailed_proteins.csv"  
test_fractions_file = "Test_Fractions.csv"
output_file = os.path.join("..", "data", "gold_data_test.csv")
# Create the gold dataset
gold_data_test = create_gold_dataset(glycans_file, proteins_file, test_fractions_file, output_file)

Reading glycans data from detailed_glycans.csv...
Read 611 glycan records
Reading proteins data from detailed_proteins.csv...
Read 52 protein records
Reading train fractions data from Test_Fractions.csv...
Read 9472 train fraction records
Merging datasets...

Merge Summary:
Total train_fractions records: 9472
Records with missing glycans: 0
Records with missing proteins: 0
Successfully merged records: 9472

Performing one-hot encoding on categorical features...
One-hot encoding Protein_Accession with 15 unique values
One-hot encoding Protein_Uniprot with 15 unique values
One-hot encoding Protein_Description with 14 unique values
One-hot encoding Protein_Amino Acid Sequence with 15 unique values
One-hot encoding Protein_ProteinID with 15 unique values
One-hot encoding Protein_CleanSequence with 15 unique values
One-hot encoding Protein_PredictedSecondaryStructure with 3 unique values
One-hot encoding Protein_Stability with 2 unique values
One-hot encoding Protein_PredictedProteinClass w

In [4]:
gold_data_train

,Concentration,f,Glycan_MolecularWeight,Glycan_HeavyAtomCount,Glycan_RingCount,Glycan_RotatableBondCount,Glycan_HBondDonorCount,Glycan_HBondAcceptorCount,Glycan_TPSA,Glycan_LogP,...,Protein_PredictedSecondaryStructure_-_Turn,Protein_Stability_-_Stable,Protein_Stability_-_Unstable,Protein_PredictedProteinClass_-_Acidic Protein,Protein_PredictedProteinClass_-_Aromatic-rich Protein,Protein_PredictedProteinClass_-_Beta-sheet Protein,Protein_PredictedProteinClass_-_Disulfide-rich Protein,Protein_PredictedProteinClass_-_Globular Protein,Protein_PredictedProteinClass_-_Highly Stable Protein,Protein_PredictedProteinClass_-_Membrane Protein
0,0.001,0.000154,337.369,23,1,15,5,8,137.71,-2.2642,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.001,0.000082,337.369,23,1,15,5,8,137.71,-2.2642,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.001,0.000290,408.404,28,1,16,6,10,183.88,-3.0573,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.001,0.000000,378.422,26,1,16,5,8,146.58,-2.1204,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.001,0.000000,321.370,22,1,14,4,7,117.48,-1.2366,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68487,200.000,0.000401,1079.022,74,4,41,17,28,496.60,-9.7028,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68488,200.000,0.047294,3899.569,267,19,141,65,106,1823.34,-43.0262,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68489,200.000,0.003314,3168.899,217,15,115,53,86,1489.00,-34.6106,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68490,200.000,0.001464,539.579,37,2,22,7,13,208.74,-3.8119,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
gold_data_train.shape

(68492, 357)

In [8]:
# Select categorical columns
categorical_columns = gold_data_train.select_dtypes(include=['object']).columns

# Get the number of unique values for each categorical column
unique_values = {col: gold_data_train[col].nunique() for col in categorical_columns}

# Display the result
unique_values

{}

In [9]:
# Select numerical columns
numerical_columns = gold_data_train.select_dtypes(include=['number']).columns

# Get the number of unique values for each numerical column
unique_values_numerical = {col: gold_data_train[col].nunique() for col in numerical_columns}

# Display the result
{col: count for col, count in unique_values_numerical.items() if count == 1}

{}

In [10]:
# Get value counts for each categorical column with less than 100 unique values
value_counts = {col: gold_data_train[col].value_counts() for col in categorical_columns if unique_values[col] < 100}

# Display the result
for col, counts in value_counts.items():
    print(f"Value counts for {col}:")
    print(counts)
    print("\n")